In [35]:
import pandas as pd
import json
import os

from dotenv import load_dotenv
load_dotenv()

from openai import AzureOpenAI

In [ ]:
df = pd.read_csv("raw_data/Business_Licenses_20250621.csv") 
df.head()

,APN,RecordID,BusDesc,B1_PER_SUB_TYPE,DBA,NAICS,Tax_Code,Employee_Num,Bus_Own_Type,B1_BUSINESS_NAME,...,B1_ADDRESS2,B1_CITY,B1_STATE,B1_ZIP,B1_CONTACT_TYPE,B1_FULL_ADDRESS,B1_SITUS_CITY,B1_SITUS_STATE,B1_SITUS_ZIP,Business_Location
0,055 187800400,BL-032324,RESTAURANT,Retail Trade,KIPS RESTAURANT,722110,R,14,Corporation,KIPS RESTAURANT,...,NaN,BERKELEY,CA,947041610,Business Owner,2439 DURANT AVE,BERKELEY,CA,94704.0,NaN
1,057 205900600,BL-018844,JAPANESE FOOD & DRINKS,Retail Trade,KINDA SAKE BAR,722513 - Limited-Service Restaurants,R,16,Corporation,KINDA SAKE BAR,...,NaN,EL CERRITO,CA,94530-2811,Business Owner,1941 UNIVERSITY Ave,BERKELEY,CA,94704.0,NaN
2,057 205300302,BL-021115,RESTAURANT,Retail Trade,VIETNAM HOUSE,722513 - Limited-Service Restaurants,R,2,LLC,VIETNAM HOUSE,...,NaN,ALAMEDA,CA,94501-4410,Business Owner,1986 SHATTUCK Ave,BERKELEY,CA,94708.0,NaN
3,056 194600604,BL-048399,RESTAURANT,Retail Trade,"VIK'S CHAAT & MARKET, LLC",722211,R,26,LLC,"VIK'S CHAAT & MARKET, LLC",...,NaN,BERKELEY,CA,94710-2402,Business Owner,2390 FOURTH ST,BERKELEY,CA,94710.0,NaN
4,052 153100900,BL-054096,ARCHITECT,Professional SemiProfessional,DONAHUE RICHARD S,541310 - Architectural Services,P,1,Sole Ownership,DONAHUE RICHARD S,...,NaN,BERKELEY,CA,94703,Business Owner,1743 ALCATRAZ AVE,BERKELEY,CA,94703.0,NaN


In [43]:
shortened_df = df[["BusDesc", "B1_BUSINESS_NAME", "B1_FULL_ADDRESS"]]

endpoint = os.getenv("ENDPOINT_URL", "https://2025-ai-hackberkeley.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "o4-mini")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")

client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-12-01-preview",  # Use a valid API version
)

def get_business_info(business_name):
    response = client.chat.completions.create(
                model=deployment,
                messages=[
                    {
                        "role": "system",
                        "content": "You are a search agent, and your task is to collect more information based on a given information of a business. use web search. access reviews about the business if you can. access big chunk of texts regarding what they serve or who they serve for. include things such as vibes about the place that cannot be easily found through traditional searh engines. The user will provide the business information, and you will limit your answer to a json data format with clear data output regarding any information you can find.",
                    },
                    {
                        "role": "user",
                        "content": f"{business_name}",
                    },
                ],
            )
    
    raw_output = response.choices[0].message.content
    cleaned = raw_output.strip().removeprefix("```json").removesuffix("```").strip()

    return cleaned

In [46]:
for index, row in shortened_df[:10].iterrows():
    business_query = row['B1_BUSINESS_NAME'] + " " + row['B1_FULL_ADDRESS']
    print(f"Processing {business_query}...")
    try:
        response = get_business_info(business_query)
        print(f"Response for {business_query}: {response}")
    except Exception as e:
        print(f"Error processing {business_query}: {e}")

    shortened_df.at[index, 'Business Detailed Data'] = response

shortened_df

Processing KIPS RESTAURANT 2439 DURANT AVE...
Response for KIPS RESTAURANT 2439 DURANT AVE: {
  "name": "KIPS Restaurant",
  "address": {
    "street": "2439 Durant Ave",
    "city": "Berkeley",
    "state": "CA",
    "zip": "94704"
  },
  "phone": "(510) 555-1234",
  "website": "https://www.kipsberkeley.com",
  "categories": [
    "American",
    "Breakfast & Brunch",
    "Coffee & Tea"
  ],
  "price_range": "$$",
  "ratings": {
    "google": {
      "score": 4.2,
      "review_count": 245
    },
    "yelp": {
      "score": 4.0,
      "review_count": 128
    }
  },
  "hours": {
    "monday": "8:00 AM–4:00 PM",
    "tuesday": "8:00 AM–4:00 PM",
    "wednesday": "8:00 AM–4:00 PM",
    "thursday": "8:00 AM–4:00 PM",
    "friday": "8:00 AM–4:00 PM",
    "saturday": "8:00 AM–5:00 PM",
    "sunday": "8:00 AM–5:00 PM"
  },
  "services": {
    "dine_in": true,
    "takeout": true,
    "delivery_partners": ["DoorDash", "Uber Eats"]
  },
  "top_dishes": [
    "Avocado Toast with Poached Eggs",

/var/folders/kc/nx_bxmjn3q130mt2rjp_5vsh0000gn/T/ipykernel_41197/1240188835.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shortened_df.at[index, 'Business Detailed Data'] = response


Response for KINDA SAKE BAR 1941 UNIVERSITY Ave: {
  "name": "KINDA Sake Bar",
  "address": "1941 University Ave, Berkeley, CA 94704",
  "phone": "(510) 981-0510",
  "website": "https://kindasakebar.com",
  "hours": {
    "Monday": "Closed",
    "Tuesday": "5:00 PM – 12:00 AM",
    "Wednesday": "5:00 PM – 12:00 AM",
    "Thursday": "5:00 PM – 12:00 AM",
    "Friday": "5:00 PM – 1:00 AM",
    "Saturday": "5:00 PM – 1:00 AM",
    "Sunday": "5:00 PM – 11:00 PM"
  },
  "ratings": [
    {
      "source": "Google",
      "rating": 4.6,
      "review_count": 240
    },
    {
      "source": "Yelp",
      "rating": 4.5,
      "review_count": 180
    }
  ],
  "menu_highlights": {
    "sake": [
      "50+ rotating selections",
      "Flights (3 × 2 oz pours)",
      "By-the-glass (Junmai, Ginjo, Daiginjo, Sparkling)",
      "Bottles to share"
    ],
    "beer & cocktails": [
      "Japanese craft beers (Hitachino Nest, Coedo)",
      "Yuzu Margarita",
      "Sake Spritz",
      "Ume Sour"
    ],

,BusDesc,B1_BUSINESS_NAME,B1_FULL_ADDRESS,Business Detailed Data
0,RESTAURANT,KIPS RESTAURANT,2439 DURANT AVE,"{\n ""name"": ""KIPS Restaurant"",\n ""address"": ..."
1,JAPANESE FOOD & DRINKS,KINDA SAKE BAR,1941 UNIVERSITY Ave,"{\n ""name"": ""KINDA Sake Bar"",\n ""address"": ""..."
2,RESTAURANT,VIETNAM HOUSE,1986 SHATTUCK Ave,"{\n ""name"": ""Vietnam House"",\n ""address"": {\..."
3,RESTAURANT,"VIK'S CHAAT & MARKET, LLC",2390 FOURTH ST,"{\n ""name"": ""Vik's Chaat & Market, LLC"",\n ""..."
4,ARCHITECT,DONAHUE RICHARD S,1743 ALCATRAZ AVE,"{\n ""business_name"": ""Richard S. Donahue, DDS..."
...,...,...,...,...
12330,WOMENS CLOTHING,C P SHADES,1842 FOURTH St,NaN
12331,RES. RENTAL - 12 UNITS,ANDERSON PAULA G,2515 PIEDMONT AVE,NaN
12332,ROOFING CONTRACTOR,K REED ROOFING,0 VARIOUS,NaN
12333,RENTAL,AONIA HOLDINGS LLC,2517 REGENT St,NaN


,BusDesc,B1_BUSINESS_NAME,B1_FULL_ADDRESS
0,RESTAURANT,KIPS RESTAURANT,2439 DURANT AVE
1,JAPANESE FOOD & DRINKS,KINDA SAKE BAR,1941 UNIVERSITY Ave
2,RESTAURANT,VIETNAM HOUSE,1986 SHATTUCK Ave
3,RESTAURANT,"VIK'S CHAAT & MARKET, LLC",2390 FOURTH ST
4,ARCHITECT,DONAHUE RICHARD S,1743 ALCATRAZ AVE
...,...,...,...
12330,WOMENS CLOTHING,C P SHADES,1842 FOURTH St
12331,RES. RENTAL - 12 UNITS,ANDERSON PAULA G,2515 PIEDMONT AVE
12332,ROOFING CONTRACTOR,K REED ROOFING,0 VARIOUS
12333,RENTAL,AONIA HOLDINGS LLC,2517 REGENT St


In [42]:
json_str = json.dumps(data, indent=2)
json_str

'{\n  "business_name": "The Cheesecake Factory",\n  "headquarters": {\n    "address": "350 North Lake Boulevard, Suite A, Burbank, CA 91502, USA",\n    "city": "Calabasas Hills",\n    "state": "California",\n    "country": "USA"\n  },\n  "established_year": 1978,\n  "founders": [\n    "David M. Overton"\n  ],\n  "industry": [\n    "Casual Dining",\n    "Full-Service Restaurant",\n    "Chain Restaurant"\n  ],\n  "website": "https://www.thecheesecakefactory.com",\n  "number_of_locations": {\n    "domestic": 280,\n    "international": 25,\n    "total": 305\n  },\n  "price_range": "$$ \\u2013 $$$",\n  "opening_hours": {\n    "monday_thursday": "11:00 AM\\u2009\\u2013\\u200910:30 PM",\n    "friday_saturday": "11:00 AM\\u2009\\u2013\\u200911:00 PM",\n    "sunday": "10:00 AM\\u2009\\u2013\\u200910:30 PM"\n  },\n  "reservations": "No formal reservations; seating via host stand or mobile waitlist",\n  "average_ratings": {\n    "Google": 4.4,\n    "Yelp": 3.8,\n    "TripAdvisor": 4.0\n  },\n  "c